In [4]:

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from Bio.Blast import NCBIWWW
from Bio import SeqIO
import csv, os

input_file = 'EnterobactericaeaP3.fas'
output_file = 'parsed_sequences.csv'
def convertFastaToCsv(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['index', 'direction', 'sequence'])
        
        for line in infile:
            if line.startswith('>'):
                parts = line[1:].strip().split('_')
                index = parts[1] if len(parts) > 1 else '1'
                direction = parts[2] if len(parts) > 2 else parts[1]
            else:
                sequence = line.strip()
                csv_writer.writerow([index, direction, sequence])
            
            
def convertFastaFolderToCsv(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith('.fas'):
            input_file = os.path.join(input_folder, filename)
            output_file = os.path.join(output_folder, f'{filename}.csv')
            
            # Check if output file already exists
            if os.path.exists(output_file):
                output_file = os.path.join(output_folder, f'{filename}.csv')
                continue
            
            convertFastaFolderToCsv(input_file, output_file)

In [ ]:

groupGeneous = pd.read_csv('data/group/geneious/Entero grp-specific primers (GENEIOUS).csv')
groupPrimer3 = pd.read_csv('data/group/primer3/EnterobactericaeaP3.fas')